In [17]:
import numpy as np
import pandas as pd

In [18]:
# Load dataset utama (shopee.xlsx)
file_shopee = "shopee.xlsx"
df_shopee = pd.read_excel(file_shopee)

# Pastikan kolom 'Waktu Pesanan Dibuat' dalam format datetime
df_shopee['Waktu Pesanan Dibuat'] = pd.to_datetime(df_shopee['Waktu Pesanan Dibuat'])

# Buat rentang tanggal lengkap dari min hingga max dalam dataset
date_range = pd.date_range(start=df_shopee['Waktu Pesanan Dibuat'].min(), end=df_shopee['Waktu Pesanan Dibuat'].max(), freq='D')

# Buat DataFrame dengan semua tanggal yang ada dalam rentang tersebut
df_full_dates = pd.DataFrame({'Waktu Pesanan Dibuat': date_range})

# Buat daftar varian produk yang valid
variants = ['Ares', 'Apollo', 'Athena']

# Buat kombinasi lengkap dari tanggal dan varian produk untuk memastikan setiap tanggal memiliki semua varian
df_reference = pd.MultiIndex.from_product(
    [df_full_dates['Waktu Pesanan Dibuat'], variants], names=['Waktu Pesanan Dibuat', 'Nama Produk']
).to_frame(index=False)

# Cek daftar tanggal unik di dataset utama (shopee.xlsx)
unique_dates_shopee = df_shopee['Waktu Pesanan Dibuat'].dt.date.unique()

# Cek daftar tanggal unik di dataset referensi (tanggal yang seharusnya ada)
unique_dates_reference = df_full_dates['Waktu Pesanan Dibuat'].dt.date.unique()

# Cari tanggal yang ada di rentang waktu, tetapi tidak ada di dataset utama
missing_dates = set(unique_dates_reference) - set(unique_dates_shopee)

# Buat DataFrame dengan tanggal yang hilang
missing_dates_df = pd.DataFrame({'Waktu Pesanan Dibuat': list(missing_dates)})

# Buat semua kombinasi tanggal yang hilang dengan varian produk
missing_date_product_combinations = pd.MultiIndex.from_product(
    [missing_dates_df['Waktu Pesanan Dibuat'], variants], names=['Waktu Pesanan Dibuat', 'Nama Produk']
).to_frame(index=False)

# Tambahkan kolom-kolom lain dengan nilai default (dummy values)
dummy_values = {
    'Harga Awal': 0.0,
    'Harga Setelah Diskon': 0.0,
    'Jumlah': 0,
    'Total Harga Produk': 0.0,
    'Total Diskon': 0.0,
    'Diskon Dari Penjual': 0.0,
    'Diskon Dari Shopee': 0.0,
    'Jumlah Produk di Pesan': 0,
    'Voucher Ditanggung Penjual': 0.0,
    'Voucher Ditanggung Shopee': 0.0,
    'Username (Pembeli)': 'dummy_user',
    'Kota/Kabupaten': 'dummy_city',
    'Provinsi': 'dummy_province',
    'Latitude': 0.0,
    'Longitude': 0.0,
    'Marketplace': 'Shopee'
}

# Isi nilai default pada data tanggal yang hilang
for col in df_shopee.columns:
    if col not in ['Waktu Pesanan Dibuat', 'Nama Produk']:
        missing_date_product_combinations[col] = dummy_values.get(col, 0)

# Konversi ulang ke datetime sebelum penggabungan
missing_date_product_combinations['Waktu Pesanan Dibuat'] = pd.to_datetime(missing_date_product_combinations['Waktu Pesanan Dibuat'])

# Gabungkan dataset utama dengan data yang berisi tanggal yang hilang
df_fixed = pd.concat([df_shopee, missing_date_product_combinations], ignore_index=True)

# Konversi ulang ke datetime setelah concat untuk memastikan tipe data tetap benar
df_fixed['Waktu Pesanan Dibuat'] = pd.to_datetime(df_fixed['Waktu Pesanan Dibuat'])

In [19]:
df_fixed.shape

(243, 18)

In [20]:
# df_fixed.info()

In [21]:
# df_fixed['Waktu Pesanan Dibuat'] = pd.to_datetime(df_shopee['Waktu Pesanan Dibuat']).dt.date


In [22]:
# df_fixed.info()

In [23]:


# Simpan data yang sudah diperbaiki ke file Excel untuk Tableau
export_path = "missing_date_filled.xlsx"
df_fixed.to_excel(export_path, index=False)

print(f"File sudah disimpan: {export_path}")

File sudah disimpan: missing_date_filled.xlsx
